In [1]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, TimestampType, StringType, DoubleType, LongType
from pyspark.sql.functions import udf, from_json, col, explode, split
from textblob import TextBlob

In [2]:
DATABASE_HOST = os.environ.get('DB_HOST', '')
DATABASE_PORT = os.environ.get('DB_PORT', '')
DATABASE_NAME = os.environ.get('DB_NAME', '')
DATABASE_USER = os.environ.get('DB_USER', '')
DATABASE_PASS = os.environ.get('DB_PASS', '')
ENV = os.environ.get('ENV', 'dev')
KAFKA_CONNECTION_STRING = os.environ.get('KAFKA_CONNECTION_STRING', 'kafka-broker:9092')
KAFKA_TOPIC_NAME = os.environ.get('KAFKA_TOPIC_NAME', 'us-election-tweet')

ENV = 'dev' if ENV == '' else ENV
KAFKA_CONNECTION_STRING = 'kafka-broker:9092' if ENV == 'dev' else KAFKA_CONNECTION_STRING

In [3]:
spark = (
    SparkSession.builder
    .appName('5003-project')
    .master('spark://spark-master:7077')
    .config("spark.files.overwrite", "true")
    .config('spark.driver.extraClassPath', '/opt/drivers/postgresql-42.2.19.jar')
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2')
    .enableHiveSupport()
    .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/conda/envs/5003-project/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3fad912d-eb09-4751-9452-8020ad743737;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.1.2/spark-sql-kafka-0-10_2.12-3.1.2.jar ...
	[SUCCESSFUL ] org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2!spark-sql-

In [4]:
schema = (
    StructType()
    .add("created_at", TimestampType(), True)
    .add("tweet_id", DoubleType(), True)
    .add("tweet", StringType(), True)
    .add("likes", DoubleType(), True)
    .add("retweet_count", DoubleType(), True)
    .add("source", StringType(), True)
    .add("user_id", DoubleType(), True)
    .add("user_name", StringType(), True)
    .add("user_screen_name", StringType(), True)
    .add("user_description", StringType(), True)
    .add("user_join_date", TimestampType(), True)
    .add("user_followers_count", DoubleType(), True)
    .add("user_location", StringType(), True)
    .add("lat", DoubleType(), True)
    .add("long", DoubleType(), True)
    .add("city", StringType(), True)
    .add("country", StringType(), True)
    .add("continent", StringType(), True)
    .add("state", StringType(), True)
    .add("state_code", StringType(), True)
    .add("collected_at", TimestampType(), True)
    .add("person", StringType(), True)
    .add("time_to_sleep", DoubleType(), True)
)

In [5]:
df = (spark
    .read
    .format("kafka")
    .option("kafka.bootstrap.servers", KAFKA_CONNECTION_STRING)
    .option("subscribe", KAFKA_TOPIC_NAME)
    .option("startingOffsets", "earliest")
    .load()
    .select(from_json(col("value").cast("string"), schema).alias("data"))
    .select("data.*")
)

In [6]:
df.take(1)

[]

In [6]:
def get_polarity(tweet):
    return TextBlob(tweet).sentiment.polarity

udf_get_polarity = udf(get_polarity, DoubleType())

In [7]:
df_with_sentiment = handle_newline.withColumn("polarity", get_polarity(handle_newline.tweet))

TypeError: The `text` argument passed to `__init__(text)` must be a string, not <class 'pyspark.sql.column.Column'>

In [6]:
query = df \
    .write \
    .format("console") \
    .start() \
    .awaitTermination()

21/11/17 10:13:58 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-4d0d804d-3772-4641-b7ca-8129ff0e8c27. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+
|                data|
+--------------------+
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
+--------------------+
only showing top 20 rows



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+
|                data|
+--------------------+
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
+--------------------+
only showing top 20 rows

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+
|                data|
+--------------------+
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...

KeyboardInterrupt: 

-------------------------------------------
Batch: 12
-------------------------------------------
+--------------------+
|                data|
+--------------------+
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
+--------------------+
only showing top 20 rows

-------------------------------------------
Batch: 13
-------------------------------------------
+--------------------+
|                data|
+--------------------+
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":.

-------------------------------------------
Batch: 142
-------------------------------------------
+--------------------+
|                data|
+--------------------+
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
|"{\"created_at\":...|
+--------------------+
only showing top 20 rows



In [ ]:
def postgres_sink(df, batch_id):
    url = f'jdbc:postgresql://{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}?user={DATABASE_USER}&password={DATABASE_PASS}'
    df.write.jdbc(url, 'sentiment', mode='append')
    
write_table = (df
    .writeStream
    .outputMode("append") 
    .foreachBatch(postgres_sink)
    .start()
    .awaitTermination())